# Westwood et al. (2022) Replication in R - Part 2: Core Analysis & Figure 2

**IMPORTANT:** Change runtime to R: Runtime -> Change runtime type -> R

This notebook replicates **Figure 2** - the key finding showing
engaged vs disengaged respondents' support for violence.

Code closely follows the original **preprocess1.R**, **preprocess2.R**, and **figure2.R**.

## Step 1: Setup

In [ ]:
# Install and load packages
install.packages(c("dplyr", "readr", "ggplot2", "cowplot", "forcats", "gtools"), quiet=TRUE)

suppressPackageStartupMessages({
  library(dplyr)
  library(readr)
  library(ggplot2)
  library(cowplot)
  library(forcats)
  library(gtools)
})

# Helper functions from functions.R
lower_ci <- function(mean, se, n, conf_level = 0.95) {
  mean - qt(1 - ((1 - conf_level) / 2), n - 1) * se
}

upper_ci <- function(mean, se, n, conf_level = 0.95) {
  mean + qt(1 - ((1 - conf_level) / 2), n - 1) * se
}

cat("Setup complete\n")

## Step 2: Download and Load Data

In [ ]:
# Download from Google Drive
download_gdrive <- function(file_id, destfile) {
  url <- paste0("https://drive.google.com/uc?export=download&id=", file_id)
  download.file(url, destfile, quiet = TRUE, mode = "wb")
}

download_gdrive("1gKIY11FaM5RmhhXTKx3wVcwGkMoTyTUM", "/tmp/study14.csv")
download_gdrive("1VfZM3hSDzIIIVp2AUGC-RwOy-Fk2t_Fm", "/tmp/study25.csv")

data14 <- read_csv("/tmp/study14.csv", show_col_types = FALSE)
data25 <- read_csv("/tmp/study25.csv", show_col_types = FALSE)

cat("Data loaded\n")
cat("study14.csv: n =", nrow(data14), "\n")
cat("study25.csv: n =", nrow(data25), "\n")

## Step 3: Preprocess Study 1

This follows **preprocess1.R** lines 1-165.

Study 1 uses a **car-ramming vignette** with two stories:
- Story 1 (partisantreatment=1): Republican driver in Florida
- Story 2 (partisantreatment=2): Democrat driver in Oregon

In [ ]:
# Filter by gc (preprocess1.R lines 4-6)
data <- data14 %>% filter(gc == 1)

# Recode party ID (preprocess1.R lines 13-16)
data$Q10[data$Q11 == "Democratic Party"] <- "Democrat"
data$Q10[data$Q11 == "Republican Party"] <- "Republican"
data$pid <- data$Q10
data$pid <- as.factor(data$pid)

# Recode experiment column (preprocess1.R line 35)
data$experiment <- recode(data$experiment, "1" = "Vignette", "2" = "Sentencing")

# Filter to Study 1 - Vignette experiment (preprocess1.R line 121)
study1 <- data[data$experiment == "Vignette", ]

# ENGAGEMENT CHECK (preprocess1.R lines 127-129)
# Story 1 asks about Florida, Story 2 asks about Oregon
study1$passed <- "Disengaged Respondent"
study1$passed[study1$Q43 == "Florida" & study1$partisantreatment == 1] <- "Engaged Respondent"
study1$passed[study1$Q49 == "Oregon" & study1$partisantreatment == 2] <- "Engaged Respondent"

# Recode DVs (preprocess1.R lines 135-148)
# Different questions for each story
study1$supportactions <- NA
study1$supportactions[study1$partisantreatment == 1] <- study1$Q44[study1$partisantreatment == 1]
study1$supportactions[study1$partisantreatment == 2] <- study1$Q50[study1$partisantreatment == 2]
study1$supportactions <- recode(study1$supportactions, 
  "Strongly support" = 5, "Support" = 4, "Neither support nor oppose" = 3, 
  "Oppose" = 2, "Strongly oppose" = 1)

study1$justified <- NA
study1$justified[study1$partisantreatment == 1] <- study1$Q45[study1$partisantreatment == 1]
study1$justified[study1$partisantreatment == 2] <- study1$Q51[study1$partisantreatment == 2]
study1$justified <- recode(study1$justified, "Justified" = 1, "Unjustified" = 0)

study1$charged <- NA
study1$charged[study1$partisantreatment == 1] <- study1$Q46[study1$partisantreatment == 1]
study1$charged[study1$partisantreatment == 2] <- study1$Q52[study1$partisantreatment == 2]
study1$charged <- recode(study1$charged, "Yes" = 1, "No" = 0)

# Alignment (preprocess1.R lines 150-164)
study1$alignment <- NA
study1$alignment[study1$version == 1 & study1$partisantreatment == 2 & study1$pid == "Democrat"] <- "Out-Party Driver"
study1$alignment[study1$version == 1 & study1$partisantreatment == 1 & study1$pid == "Democrat"] <- "In-Party Driver"
study1$alignment[study1$version == 1 & study1$partisantreatment == 2 & study1$pid == "Republican"] <- "In-Party Driver"
study1$alignment[study1$version == 1 & study1$partisantreatment == 1 & study1$pid == "Republican"] <- "Out-Party Driver"
# Version 2 = Apolitical (both stories)
study1$alignment[study1$version == 2] <- "Apolitical Driver"

study1$alignment <- as.factor(study1$alignment)

cat("Study 1 preprocessed: n =", nrow(study1), "\n")
cat("\nEngagement status:\n")
print(table(study1$passed))

## Step 4: Preprocess Study 2

This follows **preprocess2.R** lines 1-163.

Study 2 uses a **shooting vignette** with one story:
- Version 1 = Democrat shooter
- Version 2 = Republican shooter
- Version 3 = Apolitical shooter

**Key difference from Study 1:** No `partisantreatment` - just `version` determines shooter party.

In [ ]:
# Filter by gc (preprocess2.R lines 3-5)
data2 <- data25 %>% filter(gc == 1)

# Recode party ID (preprocess2.R lines 7-11)
data2$Q10[data2$Q11 == "Democratic Party"] <- "Democrat"
data2$Q10[data2$Q11 == "Republican Party"] <- "Republican"
data2$pid <- data2$Q10
data2$pid <- as.factor(data2$pid)

# Recode experiment column (preprocess2.R line 30)
data2$experiment <- recode(data2$experiment, "1" = "Vignette (Rep)", "2" = "Expressiveness")

# Filter to Study 2 - Vignette experiment (preprocess2.R line 114)
study2 <- data2[data2$experiment == "Vignette (Rep)", ]

# ENGAGEMENT CHECK (preprocess2.R lines 117-118)
# Q43 asks about Iowa (where shooting took place)
study2$passed <- "Disengaged Respondent"
study2$passed[study2$Q43 == "Iowa"] <- "Engaged Respondent"

# Recode DVs (preprocess2.R lines 125-139)
# Single set of questions (Q44, Q45, Q46)
study2$supportactions <- study2$Q44
study2$supportactions <- recode(study2$supportactions,
  "Strongly support" = 5, "Support" = 4, "Neither support nor oppose" = 3,
  "Oppose" = 2, "Strongly oppose" = 1)

study2$justified <- study2$Q45
study2$justified <- recode(study2$justified, "Justified" = 1, "Unjustified" = 0)

study2$charged <- study2$Q46
study2$charged <- recode(study2$charged, "Yes" = 1, "No" = 0)

# Alignment (preprocess2.R lines 141-150)
# Version determines shooter party directly
study2$alignment <- NA
# Version 1 = Democrat shooter
study2$alignment[study2$version == 1 & study2$pid == "Democrat"] <- "Out-Party Shooter"
study2$alignment[study2$version == 1 & study2$pid == "Republican"] <- "In-Party Shooter"
# Version 2 = Republican shooter
study2$alignment[study2$version == 2 & study2$pid == "Democrat"] <- "In-Party Shooter"
study2$alignment[study2$version == 2 & study2$pid == "Republican"] <- "Out-Party Shooter"
# Version 3 = Apolitical (note: original code has typo "Apoltical")
study2$alignment[study2$version == 3] <- "Apolitical Shooter"

study2$alignment <- as.factor(study2$alignment)

cat("Study 2 preprocessed: n =", nrow(study2), "\n")
cat("\nEngagement status:\n")
print(table(study2$passed))

## Step 5: Calculate Statistics for Figure 2

From **figure2.R** lines 7-23.

In [ ]:
# Study 1 - Justified (figure2.R lines 7-14)
plot1ajustifieddata <- study1 %>%
  filter(!is.na(alignment)) %>%
  group_by(alignment, passed) %>%
  summarise(smean = mean(justified, na.rm = TRUE),
            ssd = sd(justified, na.rm = TRUE),
            count = n(), .groups = 'drop') %>%
  mutate(se = ssd / sqrt(count),
         lower = lower_ci(smean, se, count),
         upper = upper_ci(smean, se, count))

# Study 2 - Justified (figure2.R lines 16-23)
plot1bjustifieddata <- study2 %>%
  filter(!is.na(alignment)) %>%
  group_by(alignment, passed) %>%
  summarise(smean = mean(justified, na.rm = TRUE),
            ssd = sd(justified, na.rm = TRUE),
            count = n(), .groups = 'drop') %>%
  mutate(se = ssd / sqrt(count),
         lower = lower_ci(smean, se, count),
         upper = upper_ci(smean, se, count))

cat("\nStudy 1 Statistics:\n")
print(plot1ajustifieddata)

cat("\nStudy 2 Statistics:\n")
print(plot1bjustifieddata)

## Step 6: Reproduce FIGURE 2

The key result showing engaged vs disengaged support for violence.

From **figure2.R** lines 45-70.

In [ ]:
# Merge data for plotting
plot1ajustifieddata$study <- "Study 1\n(Qualtrics)"
plot1bjustifieddata$study <- "Study 2\n(Qualtrics)"
plot_data <- bind_rows(plot1ajustifieddata, plot1bjustifieddata)

# Create Figure 2 (adapted from figure2.R)
plot1bjustified <- ggplot(plot_data, aes(x = smean, y = alignment, color = passed)) +
  ggtitle("Suspect is Justified") +
  geom_errorbarh(height = 0, aes(xmin = lower, xmax = upper), 
                 position = position_dodge(width = 0.5)) +
  geom_point(size = 6, shape = 21, fill = "white", 
             position = position_dodge(width = 0.5)) +
  geom_text(size = 3, aes(label = sprintf("%.2f", round(smean, 2))),
            position = position_dodge(width = 0.5), vjust = -1.5) +
  scale_color_manual(values = c("Disengaged Respondent" = "#D55E00", 
                                "Engaged Respondent" = "#0072B2")) +
  theme_bw() +
  theme(axis.text.y = element_text(size = 10),
        axis.text.x = element_text(size = 10),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        legend.position = "bottom") +
  guides(color = guide_legend("", override.aes = list(size = 4))) +
  xlab("Proportion Saying Suspect is Justified (95% CI)") +
  xlim(-0.01, 0.6) +
  ylab("") +
  facet_wrap(~study, ncol = 1, scales = "free_y")

print(plot1bjustified)

## Step 7: Key Results Summary

In [ ]:
cat("\n=== KEY RESULTS ===")
cat("\n\nStudy 1 - Justified (Political treatments only):\n")
study1 %>%
  filter(!grepl('Apolitical', alignment)) %>%
  group_by(passed) %>%
  summarise(prop_justified = round(mean(justified, na.rm=TRUE), 4), n=n()) %>%
  print()

cat("\nStudy 2 - Justified (Political treatments only):\n")
study2 %>%
  filter(!grepl('Apolitical', alignment)) %>%
  group_by(passed) %>%
  summarise(prop_justified = round(mean(justified, na.rm=TRUE), 4), n=n()) %>%
  print()

cat("\n=== INTERPRETATION ===")
cat("\nEngaged respondents: ~10-12% say violence is justified")
cat("\nDisengaged respondents: ~35-40% say violence is justified")
cat("\nRatio: 3-8x inflation from satisficing\n")

## Interpretation

**Figure 2 shows the key finding:**

- **Blue points (Engaged)**: ~10-12% say violence is justified
- **Orange points (Disengaged)**: ~35-40% say violence is justified

This 3-8x gap is consistent across both studies and all treatment conditions.

Prior research that averaged all respondents together got inflated estimates (~20%)
because disengaged respondents were clicking randomly or satisficing.

**Next:** Run notebook 03 for partial identification bounds